In [2]:
!pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl.metadata (14 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   - -------------------------------------- 11.3/375.7 MB 58.6 MB/s eta 0:00:07
   -

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import os
import numpy as np
import pandas as pd

# Ruta de salida
output_dir = 'resultadosKeras'
os.makedirs(output_dir, exist_ok=True)

# Paths
train_dir = 'datasetHoja/train'
test_dir = 'datasetHoja/val'

# Parámetros
img_size = (224, 224)
batch_size = 32

# Preprocesamiento
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Modelo base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50
)

# Guardar modelo y logs
model.save(os.path.join(output_dir, 'modelo_mobilenetv2.h5'))
pd.DataFrame(history.history).to_csv(os.path.join(output_dir, 'training_log.csv'), index=False)

# Evaluación
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

loss, acc = model.evaluate(test_generator)
print(f"Test accuracy: {acc:.2f}")

# Predicciones
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

results_df = pd.DataFrame({
    'Filename': test_generator.filenames,
    'TrueLabel': [class_labels[i] for i in true_classes],
    'PredictedLabel': [class_labels[i] for i in predicted_classes]
})

results_df.to_csv(os.path.join(output_dir, 'predicciones.csv'), index=False)


Found 15543 images belonging to 7 classes.
Found 3883 images belonging to 7 classes.
Epoch 1/50


C:\Users\Guille\anaconda3\envs\yolov11\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


486/486 ━━━━━━━━━━━━━━━━━━━━ 130s 261ms/step - accuracy: 0.6771 - loss: 0.8602 - val_accuracy: 0.6709 - val_loss: 0.7998
Epoch 2/50
486/486 ━━━━━━━━━━━━━━━━━━━━ 124s 255ms/step - accuracy: 0.8065 - loss: 0.4943 - val_accuracy: 0.6850 - val_loss: 0.7981
Epoch 3/50
486/486 ━━━━━━━━━━━━━━━━━━━━ 124s 254ms/step - accuracy: 0.8353 - loss: 0.4340 - val_accuracy: 0.6753 - val_loss: 0.8392
Epoch 4/50
486/486 ━━━━━━━━━━━━━━━━━━━━ 124s 254ms/step - accuracy: 0.8532 - loss: 0.3746 - val_accuracy: 0.6845 - val_loss: 0.8331
Epoch 5/50
486/486 ━━━━━━━━━━━━━━━━━━━━ 124s 255ms/step - accuracy: 0.8848 - loss: 0.3118 - val_accuracy: 0.7013 - val_loss: 0.7971
Epoch 6/50
486/486 ━━━━━━━━━━━━━━━━━━━━ 124s 256ms/step - accuracy: 0.8955 - loss: 0.2742 - val_accuracy: 0.6858 - val_loss: 0.8970
Epoch 7/50
486/486 ━━━━━━━━━━━━━━━━━━━━ 125s 257ms/step - accuracy: 0.9072 - loss: 0.2341 - val_accuracy: 0.6971 - val_loss: 0.9475
Epoch 8/50
486/486 ━━━━━━━━━━━━━━━━━━━━ 124s 255ms/step - accuracy: 0.9209 - loss: 0.20

Found 5125 images belonging to 7 classes.
5125/5125 ━━━━━━━━━━━━━━━━━━━━ 94s 18ms/step - accuracy: 0.8913 - loss: 0.7932
Test accuracy: 0.85
5125/5125 ━━━━━━━━━━━━━━━━━━━━ 91s 18ms/step
